### Мы не брали во внимание сводный анализ в силу 

In [ ]:
import pandas as pd
import numpy as np
import camelot
import os
import re
import json
import time
import pathlib

## Тут мы написали функции препроцессинга

In [188]:
def preprocessor(table: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess table
    """
    table_columns = table.iloc[1]
    table.columns = table_columns
    table.columns.name = None
    table = table.drop(index=[0, 1])
    table = table.reset_index(drop=True)
    return (pd.DataFrame(table).iloc[:, :5])

In [117]:
tables = camelot.read_pdf(
                    '../data/ЭНА/Рацион мтф высокое 26.06.25_ЭНАЛБ.pdf',
                    flavor='lattice'
                )
data = tables[0]
data_preproc = preprocessor(data.df)
tmp = data_preproc[1]['Ингредиенты'].map(clean_ingredient)
data_preproc[1]['Ингредиенты'] = tmp

регулярками приводим строки к одному виду

In [189]:
RE_DATE = re.compile(r"\b\d{1,2}[./-]\d{1,2}[./-]\d{2,4}\b")
RE_LONG_CODE = re.compile(r"\b(?:\d{1,4}\.){2,}\d{1,4}\b") 
RE_YEAR = re.compile(r"\b(19|20)\d{2}\b")
RE_TRAIL_DOTNUM = re.compile(r"(?<=\s)[\.\-]\d{1,3}\b")    
RE_PERCENT_TAIL = re.compile(r"\d+\s?%.*$")                
RE_MULTI_SPACE = re.compile(r"\s+")
RE_SP_BEFORE_PUNCT = re.compile(r"\s+([,.;:])")
RE_PUNCT_DUP = re.compile(r"[,.]{2,}")

TAILKEYS = r"(суха[яй]|мелк|помол|сечк|ток\d*|гранул|экструд|смесь|мешан|энпкх|энанпкх|энапкх|эн|энк)"
RE_TAIL_AFTER_COMMA = re.compile(rf",\s*.*?(?=($|\b))", re.IGNORECASE)
RE_TAILKEYS_AFTER_COMMA = re.compile(rf",\s*(?={TAILKEYS})[^\n]*", re.IGNORECASE)

In [190]:
def clean_ingredient(s: str) -> str:
    """
    Clear ingredients with regulat expressions
    """
    if not isinstance(s, str):
        s = "" if pd.isna(s) else str(s)

    s = s.replace("\n", " ").strip()

    s = re.sub(r"\bкомбиком\b", "комбикорм", s, flags=re.IGNORECASE)

    masks = {}
    # оставляем комбикормы номера
    def _mask(match):
        key = f"__MASKNO_{len(masks)}__"
        masks[key] = match.group(0)
        return key
    s = re.sub(r"№\s*\d+", _mask, s) 

    s = RE_DATE.sub("", s)
    s = RE_LONG_CODE.sub("", s)
    s = RE_YEAR.sub("", s)
    s = RE_TRAIL_DOTNUM.sub("", s)

    s = RE_PERCENT_TAIL.sub("", s)

    s = RE_TAILKEYS_AFTER_COMMA.sub("", s)

    s = s.replace("/", " ")

    s = RE_SP_BEFORE_PUNCT.sub(r"\1", s)
    s = RE_PUNCT_DUP.sub(lambda m: m.group(0)[0], s)
    s = RE_MULTI_SPACE.sub(" ", s).strip(" ,.;:")

    for key, val in masks.items():
        s = s.replace(key, val)

    s = s.rstrip(".,;: ").strip()

    return s

## Наши культуры и ингридиенты

In [120]:
with open('cultures.json', 'r') as f:
    cultures = json.load(f)
cultures = pd.DataFrame.from_dict(cultures)

In [121]:
cultures = cultures.drop(columns=['code', 'name_en'])

In [122]:
cultures.head()

,name_ru,type
0,Сенаж,feed_form
1,Силос,feed_form
2,Сено,feed_form
3,Солома,feed_form
4,Концентраты,feed_form


### Наша общая табличка

## Тут мы начинаем парсить таблички с ингридиентами

In [ ]:
def main(table: pd.DataFrame, columns: list):
    """
    Parsing tables
    """
    path = pathlib.Path('../data')
    dirs = [x for x in path.iterdir() if x.is_dir()]
    columns = ['Рецепты'] + columns
        
    for d in dirs:
        files =  [x for x in d.iterdir() if x.suffix == '.pdf']
        for f in files:
            try:
                tables = camelot.read_pdf(
                    f,
                    flavor='lattice'
                )
                ration_name = os.path.splitext(os.path.basename(f))[0]

                data = tables[0]
                data_preproc = preprocessor(data.df)
                data_preproc['Ингредиенты'] = data_preproc['Ингредиенты'].map(clean_ingredient)
                data_preproc = data_preproc.drop_duplicates(subset=['Ингредиенты'], keep='first')
                
                # преобразуем в одну строку и подгоняем под pivot talbe
                stacked = data_preproc.set_index('Ингредиенты').stack()
                wide_row = stacked.to_frame().T
                wide_row.columns = [f"{ingredient} {metric}" for ingredient, metric in wide_row.columns]

                wide_row = wide_row.reindex(columns=columns, fill_value=0)
                wide_row['Рецепты'] = ration_name
                table = pd.concat([table, wide_row], ignore_index=True)
                
            except Exception as e:
                print(f"Ошибка при обработке файла {f}: {e}")
                time.sleep(0.5)  
                continue
    
    print("Обработка завершена успешно!")
    return table


In [ ]:
import json
with open('../data/cultures.json', 'r') as f:
    data = json.load(f)
    df = pd.DataFrame.from_dict(data)
    columns = [f"{ingredient} {col}" for ingredient in df.name_ru for col in ['СВ %','ГП кг','СВ кг', '% ГП','% СВ']]
    cols = pd.Series(columns)
    cols.to_pickle('../data/columns')

columns = pd.read_pickle('../data/columns')
print(columns.to_list())

['Сенаж СВ %', 'Сенаж ГП кг', 'Сенаж СВ кг', 'Сенаж % ГП', 'Сенаж % СВ', 'Силос СВ %', 'Силос ГП кг', 'Силос СВ кг', 'Силос % ГП', 'Силос % СВ', 'Сено СВ %', 'Сено ГП кг', 'Сено СВ кг', 'Сено % ГП', 'Сено % СВ', 'Солома СВ %', 'Солома ГП кг', 'Солома СВ кг', 'Солома % ГП', 'Солома % СВ', 'Концентраты СВ %', 'Концентраты ГП кг', 'Концентраты СВ кг', 'Концентраты % ГП', 'Концентраты % СВ', 'Консервированное зерно СВ %', 'Консервированное зерно ГП кг', 'Консервированное зерно СВ кг', 'Консервированное зерно % ГП', 'Консервированное зерно % СВ', 'Корнаж СВ %', 'Корнаж ГП кг', 'Корнаж СВ кг', 'Корнаж % ГП', 'Корнаж % СВ', 'Сенаж в плёнке СВ %', 'Сенаж в плёнке ГП кг', 'Сенаж в плёнке СВ кг', 'Сенаж в плёнке % ГП', 'Сенаж в плёнке % СВ', 'Люцерна СВ %', 'Люцерна ГП кг', 'Люцерна СВ кг', 'Люцерна % ГП', 'Люцерна % СВ', 'Эспарцет СВ %', 'Эспарцет ГП кг', 'Эспарцет СВ кг', 'Эспарцет % ГП', 'Эспарцет % СВ', 'Многолетние травы СВ %', 'Многолетние травы ГП кг', 'Многолетние травы СВ кг', 'Многолет

In [205]:
new_table = pd.DataFrame(columns=columns)
reciept = pd.DataFrame(columns=['Рецепты'])
table = pd.concat([reciept, new_table], axis=0)
table

,Рецепты,Сенаж СВ %,Сенаж ГП кг,Сенаж СВ кг,Сенаж % ГП,Сенаж % СВ,Силос СВ %,Силос ГП кг,Силос СВ кг,Силос % ГП,...,Комбикорм №14 СВ %,Комбикорм №14 ГП кг,Комбикорм №14 СВ кг,Комбикорм №14 % ГП,Комбикорм №14 % СВ,Комбикорм №15 СВ %,Комбикорм №15 ГП кг,Комбикорм №15 СВ кг,Комбикорм №15 % ГП,Комбикорм №15 % СВ


In [206]:
table = main(table, columns)


/Users/aleksandrandreev/EcoNiva-Cows/venv/lib/python3.13/site-packages/camelot/parsers/base.py:124: UserWarning: page-1 is image-based, camelot only works on text-based pages.
  warnings.warn(


Ошибка при обработке файла ../data/ЭНА/Рацион Д1 ЖК Озерки 24.04.2024 ЭНАЮ.pdf: list index out of range
Обработка завершена успешно!


### Полученная табличка(много нулей, поэтому pivot table)

In [ ]:
table

,Рецепты,Сенаж СВ %,Сенаж ГП кг,Сенаж СВ кг,Сенаж % ГП,Сенаж % СВ,Силос СВ %,Силос ГП кг,Силос СВ кг,Силос % ГП,...,Комбикорм №14 СВ %,Комбикорм №14 ГП кг,Комбикорм №14 СВ кг,Комбикорм №14 % ГП,Комбикорм №14 % СВ,Комбикорм №15 СВ %,Комбикорм №15 ГП кг,Комбикорм №15 СВ кг,Комбикорм №15 % ГП,Комбикорм №15 % СВ
0,Отчет_Д1 ЖК Колыбелка 24.04.2024_ЭНАПБ,"35,700","9,804","3,500","22,73",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Рацион Д1 ЖК Коршево 26.06.25_ЭНАС,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Рацион МТФ Щучье 26.06.2025_ЭНАПБ,"34,70","10,66","3,70","26,16",0,"39,80","17,59","7,00","43,16",...,0,0,0,0,0,0,0,0,0,0
3,Рацион мтф высокое 25.02.25_ЭНАЛБ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ЖК Бобров-2 Д1 (КК-10) 25.02.25(2)_ЭНАВ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Отчет_Д1 Аристово 14.04.25_КНВ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89,Отчет_Д1 МТФ Романово (24.04.24)_КНЗ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90,Отчет_ЖК Богданино Д1 18.06.25_v1_КНЮ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,Отчет_Д1 Л=1 Гусево 11.07.25 (2)_КНЗ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [209]:
table.to_excel('full_table.xlsx')

In [1]:
table.head()

NameError: name 'table' is not defined